# Import

In [3]:
import matplotlib
import pylab as plt
import numpy as np
import pandas as pd 
from numpy import random

In [4]:
# Read data file
df= pd.read_csv("..\GHG_Emission.csv",na_values=['NA', '?','']) 
np.random.seed(42)

#Rearrange the index order to a random permutation of exising index
GHG = df.reindex(np.random.permutation(df.index))

# Reset index
GHG.reset_index(inplace=True, drop=True)

# Display top five rows
GHG.head()

,X Coordinate (km),Y Coordinate (km),Measured Depth (m),Deviation (deg),Abandoned (True/False),Surface-Casing Weight (kg/m),Production-Casing Size (mm),Cumulative GAS Prod. (e3m3),Month Well Spudded,Classification,Emission Rate (m3/day)
0,588.1,476.3,NaN,NaN,True,NaN,177.8,32683.5,0.0,Non Serious,44.432680
1,62.4,666.9,491.7,NaN,False,35.7,139.7,NaN,0.0,Non Serious,29.998576
2,534.4,391.8,NaN,15.690192,True,NaN,177.8,32683.5,0.0,Serious,55.424137
3,298.7,583.0,NaN,NaN,True,35.7,139.7,32683.6,0.0,Serious,53.076994
4,513.8,434.9,2598.2,9.273310,False,NaN,114.3,32683.6,843.0,Serious,50.506939


In [30]:
#Statistical Analysis of the data
GHG.describe()

,X Coordinate (km),Y Coordinate (km),Measured Depth (m),Deviation (deg),Surface-Casing Weight (kg/m),Production-Casing Size (mm),Cumulative GAS Prod. (e3m3),Month Well Spudded,Emission Rate (m3/day)
count,1500.000000,1500.000000,1071.000000,556.000000,1140.000000,1392.000000,1148.000000,1500.000000,1500.000000
mean,445.051933,489.247133,1421.464426,31.400825,41.920000,144.073420,32683.537195,177.501533,50.161205
std,174.153002,218.796603,943.375914,18.960863,12.611025,26.083219,0.093194,248.301182,9.912002
min,10.900000,1.400000,158.600000,0.231718,13.700000,73.000000,32683.300000,0.000000,11.750512
25%,339.300000,368.100000,674.150000,14.642007,35.700000,114.300000,32683.500000,0.000000,43.476749
50%,509.250000,494.000000,1102.800000,31.660038,35.700000,139.700000,32683.500000,0.000000,50.262507
75%,592.600000,595.325000,1912.900000,44.343357,48.100000,177.800000,32683.600000,321.000000,56.689195
max,649.600000,1186.100000,5418.900000,76.894237,107.200000,244.500000,32683.800000,1264.000000,78.843781


In [31]:
GHG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   X Coordinate (km)             1500 non-null   float64
 1   Y Coordinate (km)             1500 non-null   float64
 2   Measured Depth (m)            1071 non-null   float64
 3   Deviation (deg)               556 non-null    float64
 4   Abandoned (True/False)        1500 non-null   bool   
 5   Surface-Casing Weight (kg/m)  1140 non-null   float64
 6   Production-Casing Size (mm)   1392 non-null   float64
 7   Cumulative GAS Prod. (e3m3)   1148 non-null   float64
 8   Month Well Spudded            1500 non-null   float64
 9   Classification                1500 non-null   object 
 10  Emission Rate (m3/day)        1500 non-null   float64
dtypes: bool(1), float64(9), object(1)
memory usage: 118.8+ KB


From a very quick look at the data we notice a lot of missing values

# Data Processing

## Removing Outliers

Emission Rate values outside the range of 𝜇±2.5𝜎 (𝜇 = mean, 𝜎 = standard deviation) are considered outliers and are removed

In [32]:
def outlier_remove(df, n,name):
    """Delete rows for a specified column where values are out of +/- n*sd standard deviations
    df  : Pandas dataframe
    n   : n in the equation 𝑚±𝑛𝜎
    name: Column name
    """
    mean=df[name].mean() # Calclute mean of column
    sd=df[name].std()    # Calclute standard deviation of column
    drop_r = df.index[(mean -n * sd> df[name]) | (mean+n * sd< df[name])]
    df.drop(drop_r, axis=0, inplace=True)
    df.reset_index(inplace=True, drop=True) # Reset index

In [33]:
# Drop outliers in last column 'Emission Rate (m3/day)'
outlier_remove(GHG, n=2.5,name='Emission Rate (m3/day)') 
GHG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   X Coordinate (km)             1478 non-null   float64
 1   Y Coordinate (km)             1478 non-null   float64
 2   Measured Depth (m)            1056 non-null   float64
 3   Deviation (deg)               552 non-null    float64
 4   Abandoned (True/False)        1478 non-null   bool   
 5   Surface-Casing Weight (kg/m)  1122 non-null   float64
 6   Production-Casing Size (mm)   1371 non-null   float64
 7   Cumulative GAS Prod. (e3m3)   1130 non-null   float64
 8   Month Well Spudded            1478 non-null   float64
 9   Classification                1478 non-null   object 
 10  Emission Rate (m3/day)        1478 non-null   float64
dtypes: bool(1), float64(9), object(1)
memory usage: 117.0+ KB


# Classification

For Classification, Class 0 is assigned to 'Non Serious' emissions and Class 1 is assigned to 'Serious'. Also, we can drop 'Emission Rate (m3/day)'

In [34]:
#Replace Non Serious as 0 and Serious as 1
GHG['Classification']=GHG['Classification'].replace('Non Serious', 0)
GHG['Classification']=GHG['Classification'].replace('Serious', 1)

#Drop 'Emission Rate (m3/day)' column
GHG=GHG.drop('Emission Rate (m3/day)',axis=1)

## Split the data

Data is divided into a 80/20 split of training set and test set 

In [35]:
# A 80/20 split of training and test sets using stratified shuffle split
from sklearn.model_selection import StratifiedShuffleSplit

spt = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in spt.split(GHG.drop('Classification',axis=1), GHG["Classification"]):
    x_train = GHG.loc[train_idx].drop('Classification',axis=1)
    x_test = GHG.loc[test_idx].drop('Classification',axis=1)
    y_train = GHG[['Classification']].loc[train_idx]
    y_test = GHG[['Classification']].loc[test_idx]
    
for dataset in (x_train, x_test, y_train, y_test):
                 dataset.reset_index(inplace=True, drop=True) # Reset index
    
x_train.head()

,X Coordinate (km),Y Coordinate (km),Measured Depth (m),Deviation (deg),Abandoned (True/False),Surface-Casing Weight (kg/m),Production-Casing Size (mm),Cumulative GAS Prod. (e3m3),Month Well Spudded
0,524.5,380.2,786.2,19.348187,True,35.7,139.7,32683.6,650.7
1,46.3,823.2,2915.5,65.623685,True,35.7,139.7,32683.7,814.2
2,637.9,460.2,1050.0,NaN,False,62.5,NaN,32683.5,209.7
3,605.3,555.2,NaN,24.413819,True,53.6,177.8,32683.3,0.0
4,594.0,447.4,1525.0,44.209760,True,48.1,177.8,32683.6,578.4


In [36]:
# Save training dataframe for visualization
pd.concat([x_train,y_train],axis=1).to_csv("./Data/classification_train_vis.csv",index=False)

## Imputation 

In [37]:
from sklearn.impute import SimpleImputer

# Imputs median
imput_mdn = SimpleImputer(strategy="median") 

In [38]:
# Select numerical columns that need to be imputed with median
numcolumns = x_train.drop('Abandoned (True/False)',axis=1)

In [39]:
# Calculate median and impute into missing values
x_train_im = imput_mdn.fit_transform(numcolumns)

In [40]:
#Create imputed dataframe
x_train_im = pd.DataFrame(x_train_im,columns=numcolumns.columns)
x_train_im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   X Coordinate (km)             1182 non-null   float64
 1   Y Coordinate (km)             1182 non-null   float64
 2   Measured Depth (m)            1182 non-null   float64
 3   Deviation (deg)               1182 non-null   float64
 4   Surface-Casing Weight (kg/m)  1182 non-null   float64
 5   Production-Casing Size (mm)   1182 non-null   float64
 6   Cumulative GAS Prod. (e3m3)   1182 non-null   float64
 7   Month Well Spudded            1182 non-null   float64
dtypes: float64(8)
memory usage: 74.0 KB


In [41]:
# Transform test dataset using median from train dataset
x_test_im = imput_mdn.transform(x_test.drop('Abandoned (True/False)',axis=1))
x_test_im = pd.DataFrame(x_test_im,columns=numcolumns.columns) 

## Text Handling

In [42]:
from sklearn.preprocessing import OneHotEncoder

#Transforms text data into multiple columns of 1 and 0
cat_encoder = OneHotEncoder()

In [43]:
#Select text columns that need to be one hot encoded
txtcolumns = x_train[['Abandoned (True/False)']]

In [44]:
#Fit & transform the data to be one hot encoded
x_train_text = cat_encoder.fit_transform(txtcolumns)
x_train_text = x_train_text.toarray()
x_train_text[0:10]

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [45]:
#Transform test data based on training data
x_test_text = cat_encoder.transform(x_test[['Abandoned (True/False)']])
x_test_text = x_test_text.toarray()

## Standardization

In [46]:
from sklearn.preprocessing import StandardScaler

#As the columns are in different units we must standardize the data
scaler = StandardScaler()

In [47]:
#Fit & Transform the data
x_train_std = scaler.fit_transform(x_train_im)
x_train_std[0:5]

array([[ 4.58248673e-01, -5.05436155e-01, -6.82011525e-01,
        -1.01939953e+00, -4.17717109e-01, -1.66186483e-01,
         8.57376975e-01,  1.88049809e+00],
       [-2.28786776e+00,  1.50437761e+00,  1.90278773e+00,
         2.95963286e+00, -4.17717109e-01, -1.66186483e-01,
         2.07689513e+00,  2.53317191e+00],
       [ 1.10946073e+00, -1.42490102e-01, -3.61779503e-01,
         3.96287431e-04,  1.91117083e+00, -1.66186483e-01,
        -3.62141177e-01,  1.20075137e-01],
       [ 9.22251617e-01,  2.88508335e-01, -2.86030912e-01,
        -5.83827428e-01,  1.13777148e+00,  1.34917663e+00,
        -2.80117748e+00, -7.17023941e-01],
       [ 8.57360116e-01, -2.00561471e-01,  2.14832364e-01,
         1.11834098e+00,  6.59828057e-01,  1.34917663e+00,
         8.57376975e-01,  1.59188453e+00]])

In [48]:
#Transform the test data
x_test_std = scaler.transform(x_test_im)

## Concatenate

In [49]:
#Concatenate all processed and scaled training data
x_train_clf = np.concatenate((x_train_std[:,:4],x_train_text,x_train_std[:,4:]),axis=1)
x_train_clf[0:5]

array([[ 4.58248673e-01, -5.05436155e-01, -6.82011525e-01,
        -1.01939953e+00,  0.00000000e+00,  1.00000000e+00,
        -4.17717109e-01, -1.66186483e-01,  8.57376975e-01,
         1.88049809e+00],
       [-2.28786776e+00,  1.50437761e+00,  1.90278773e+00,
         2.95963286e+00,  0.00000000e+00,  1.00000000e+00,
        -4.17717109e-01, -1.66186483e-01,  2.07689513e+00,
         2.53317191e+00],
       [ 1.10946073e+00, -1.42490102e-01, -3.61779503e-01,
         3.96287431e-04,  1.00000000e+00,  0.00000000e+00,
         1.91117083e+00, -1.66186483e-01, -3.62141177e-01,
         1.20075137e-01],
       [ 9.22251617e-01,  2.88508335e-01, -2.86030912e-01,
        -5.83827428e-01,  0.00000000e+00,  1.00000000e+00,
         1.13777148e+00,  1.34917663e+00, -2.80117748e+00,
        -7.17023941e-01],
       [ 8.57360116e-01, -2.00561471e-01,  2.14832364e-01,
         1.11834098e+00,  0.00000000e+00,  1.00000000e+00,
         6.59828057e-01,  1.34917663e+00,  8.57376975e-01,
         1.

In [50]:
#Concatenate all processed and scaled test data
x_test_clf = np.concatenate((x_test_std[:,:4],x_test_text,x_test_std[:,4:]),axis=1)

# Save Classification Data

In [51]:
# Concatenate training attributes and target into one dataframe
train_clf = pd.concat([pd.DataFrame(x_train_clf,columns=['X Coordinate (km)','Y Coordinate (km)', 'Measured Depth (m)',
                                                'Deviation (deg)','Abandoned (False)','Abandoned (True)',
                                                'Surface-Casing Weight (kg/m)','Production-Casing Size (mm)',
                                                'Cumulative GAS Prod. (e3m3)','Month Well Spudded']),
                        y_train],axis=1)
train_clf

,X Coordinate (km),Y Coordinate (km),Measured Depth (m),Deviation (deg),Abandoned (False),Abandoned (True),Surface-Casing Weight (kg/m),Production-Casing Size (mm),Cumulative GAS Prod. (e3m3),Month Well Spudded,Classification
0,0.458249,-0.505436,-0.682012,-1.019400,0.0,1.0,-0.417717,-0.166186,0.857377,1.880498,1
1,-2.287868,1.504378,1.902788,2.959633,0.0,1.0,-0.417717,-0.166186,2.076895,2.533172,1
2,1.109461,-0.142490,-0.361780,0.000396,1.0,0.0,1.911171,-0.166186,-0.362141,0.120075,0
3,0.922252,0.288508,-0.286031,-0.583827,0.0,1.0,1.137771,1.349177,-2.801177,-0.717024,1
4,0.857360,-0.200561,0.214832,1.118341,0.0,1.0,0.659828,1.349177,0.857377,1.591885,1
...,...,...,...,...,...,...,...,...,...,...,...
1177,-2.344720,0.744913,-0.766500,0.000396,0.0,1.0,0.103676,-0.166186,0.857377,-0.717024,1
1178,0.921103,0.093879,0.634970,0.000396,0.0,1.0,-0.417717,1.349177,-0.362141,-0.717024,0
1179,0.918232,-0.019542,-0.286031,-0.789843,0.0,1.0,-0.417717,-1.176429,-0.362141,-0.717024,1
1180,0.218782,-1.102482,0.479588,1.249682,0.0,1.0,-0.417717,-1.176429,2.076895,-0.717024,1


In [52]:
# Save training data to a csv
train_clf.to_csv("./Data/classification_train.csv",index=False)

In [53]:
# Concatenate test attributes and target into one dataframe
test_clf = pd.concat([pd.DataFrame(x_test_clf,columns=['X Coordinate (km)','Y Coordinate (km)', 'Measured Depth (m)',
                                                'Deviation (deg)','Abandoned (False)','Abandoned (True)',
                                                'Surface-Casing Weight (kg/m)','Production-Casing Size (mm)',
                                                'Cumulative GAS Prod. (e3m3)','Month Well Spudded']),
                        y_test],axis=1)

In [54]:
# Save test data to a csv
test_clf.to_csv("./Data/classification_test.csv",index=False)